### Utilizing Cryptocompare Website API for Data Mining

In [8]:
endpoint = 'https://min-api.cryptocompare.com/data/histoday'
res = requests.get(endpoint + '?fsym=BTC&tsym=CAD&limit=2000')
hist = pd.DataFrame(json.loads(res.content)['Data'])
hist = hist.set_index('time')
hist.index = pd.to_datetime(hist.index, unit='s')
target_col = 'close'

In [9]:
hist.value_counts

<bound method DataFrame.value_counts of                 high       low      open  volumefrom     volumeto     close  \
time                                                                          
2018-03-16  11363.92  10682.89  10772.45      908.75   9938473.94  11031.87   
2018-03-17  11529.53  10288.24  11031.62      713.03   7573045.24  10540.55   
2018-03-18  11055.26   9687.76  10541.91     1307.06  13247879.83  10882.76   
2018-03-19  11483.72  10767.82  10881.34     1153.03  12586272.22  11304.89   
2018-03-20  11916.17  11032.43  11304.89      607.89   6789380.22  11760.75   
...              ...       ...       ...         ...          ...       ...   
2023-09-02  35220.93  34846.44  34994.67       19.69    689285.74  35078.23   
2023-09-03  35441.36  34995.41  35078.23       21.00    738893.23  35297.26   
2023-09-04  35721.67  34897.22  35297.26       24.62    866308.32  35035.46   
2023-09-05  35351.72  34831.91  35035.46       45.67   1599588.58  35076.63   
2023-09-06  

In [21]:
from datetime import datetime
import json
import pandas as pd
import time  # Add this import statement to use time.mktime()
import requests  # Add this import statement to use requests

In [24]:

def construct_download_url(
    ticker,
    period1,
    period2,
    interval='daily'

):
    def convert_to_seconds(period):
        datetime_value = datetime.strptime(period, '%Y-%m-%d')
        total_seconds = int(time.mktime(datetime_value.timetuple())) + 86400
        return total_seconds

    try:
        interval_reference = {'daily': '1d', 'weekly': '1wk', 'monthly': '1mo'}
        _interval = interval_reference.get(interval)
        if _interval is None:
            print('interval code is incorrect')
            return
        p1 = convert_to_seconds(period1)
        p2 = convert_to_seconds(period2)
        url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={p1}&period2={p2}&interval={_interval}&filter=history'
        return url

    except Exception as e:
        print(e)
        return

#### Getting the Dataset online
query_url = construct_download_url('BTC-USD', '2019-01-01', '2023-01-01', 'daily')

# Make an HTTP GET request to download the data
response = requests.get(query_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Convert the response content to a DataFrame
    df = pd.read_csv(pd.compat.StringIO(response.text))
    df.set_index('Date', inplace=True)

    ### Saving dataset as CSV file;
    df.to_csv('crypto_price_data.csv')

    ##As JSON
 #   with open('BTC-USD.json', 'w') as f:
  #      f.write(json.dumps(df.T.to_dict(), indent=4))
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


Failed to retrieve data. Status code: 403


In [14]:
### MAIN CODE

import pandas as pd
import numpy as np

# Load cryptocurrency price data (assuming you have a CSV file)
crypto_data = hist#pd.read_csv('crypto_price_data.csv')

# Assuming your dataset has columns 'Date' and 'Close' for the date and closing prices
# Convert the 'Date' column to datetime if it's not already in that format
#crypto_data['Date'] = pd.to_datetime(crypto_data['Date'])

# Sort the data by date if it's not already sorted
crypto_data.sort_values(by='Date', inplace=True)

# Calculate Moving Averages (e.g., 7-day and 30-day moving averages)
crypto_data['7D_MA'] = crypto_data['Close'].rolling(window=7).mean()
crypto_data['30D_MA'] = crypto_data['Close'].rolling(window=30).mean()

# Calculate Rate of Change (ROC)
n = 1  # You can adjust the lookback period
crypto_data['ROC'] = (crypto_data['Close'] - crypto_data['Close'].shift(n)) / crypto_data['Close'].shift(n)

# Calculate Relative Strength Index (RSI)
def calculate_rsi(data, period=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

crypto_data['RSI'] = calculate_rsi(crypto_data)

# Drop rows with NaN values (resulting from rolling calculations)
crypto_data.dropna(inplace=True)

# Optionally, you can save the feature-engineered data to a new CSV file
# crypto_data.to_csv('crypto_price_data_with_features.csv', index=False)

# Now you can use the crypto_data DataFrame for deep learning modeling
#we load cryptocurrency price data from a CSV file, convert dates to datetime objects, sort the data by date, and then calculate various features such as moving averages (7-day and 30-day), rate of #change (ROC), and the Relative Strength Index (RSI). You can customize this code to add more features or adjust the lookback periods and parameters according to your modeling needs.

#MARKET SENTIMENTAL ANALYSIS

'''
We load both cryptocurrency price data and market news sentiment data from CSV files.

We merge the two datasets based on the publication date.

We use the VADER sentiment analysis library to calculate sentiment scores for news headlines.

We normalize the price and sentiment data to a common scale between 0 and 1 using Min-Max scaling.

We create sequences of input features and target values for the LSTM model.

We split the data into training and testing sets and build an LSTM-based deep learning model for price prediction.

The model is trained, evaluated, and used to make predictions
'''


import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import numpy as np

# Load cryptocurrency price data (assuming you have a CSV file)
crypto_data = hist#pd.read_csv('crypto_price_data.csv')

# Load market news sentiment data (assuming you have a CSV file with 'Date' and 'Headline' columns)
news_data = pd.read_csv('market_news_sentiments.csv')

# Convert the 'Date' columns to datetime objects
#crypto_data['Date'] = pd.to_datetime(crypto_data['Date'])
#news_data['Date'] = pd.to_datetime(news_data['Date'])

# Merge cryptocurrency price data with market news sentiment data
merged_data = pd.merge(crypto_data, news_data, on='Date', how='left')

# Fill missing sentiment values with neutral sentiment (0)
merged_data['Sentiment'].fillna(0, inplace=True)

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Calculate sentiment scores for news headlines
merged_data['Sentiment'] = merged_data['Headline'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Normalize price and sentiment data
scaler = MinMaxScaler()
merged_data[['Close', 'Sentiment']] = scaler.fit_transform(merged_data[['Close', 'Sentiment']])

# Define the number of past days to consider for prediction
lookback = 7

# Create sequences of input features and target (next day's closing price)
X = []
y = []
for i in range(len(merged_data) - lookback):
    X.append(merged_data['Close'].values[i:i+lookback])
    y.append(merged_data['Close'].values[i+lookback])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a deep learning model (LSTM-based in this example)
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Reshape X_train and X_test for LSTM input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

# Make predictions using the trained model
predictions = model.predict(X_test)

# Inverse transform the scaled predictions and actual values to get actual prices
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)




'''

# You can now analyze the predictions and evaluate the model's performance
In this code:

We load both cryptocurrency price data and market news sentiment data from CSV files.

We merge the two datasets based on the publication date.

We use the VADER sentiment analysis library to calculate sentiment scores for news headlines.

We normalize the price and sentiment data to a common scale between 0 and 1 using Min-Max scaling.

We create sequences of input features and target values for the LSTM model.

We split the data into training and testing sets and build an LSTM-based deep learning model for price prediction.

The model is trained, evaluated, and used to make predictions.

Please note that this is a simplified example, and more advanced models and feature engineering techniques can be 
explored for better performance. Additionally, you may want to fine-tune hyperparameters and optimize the model 
further for your specific use case.




User
why do you need normalization
ChatGPT
Normalization is a crucial preprocessing step when working with numerical data, including cryptocurrency prices and 
sentiment scores, for several reasons:

Scale Independence: Different features often have different scales. For example, cryptocurrency prices can have 
values in the thousands or tens of thousands, while sentiment scores typically range from -1 to 1. Deep learning
models, particularly gradient-based optimization algorithms, are sensitive to the scale of input features. 
Normalization ensures that all features contribute more equally to the model's training, preventing certain features from dominating the learning process.

Faster Convergence: Normalization helps the optimization process converge faster during model training.
Without normalization, the optimization algorithm may oscillate or take longer to find an optimal solution,
especially if the features have significantly different scales.

Improved Generalization: Normalization can lead to better model generalization. When features are normalized to a 
common scale, the model is less likely to overfit to the training data and is better equipped to make accurate
predictions on unseen data.

Stable Gradient Descent: Gradient descent, a common optimization technique used in deep learning, relies on the 
magnitude of gradients to update model parameters. If features have different scales, the gradients can vary widely, leading to unstable updates and convergence issues. Normalization helps stabilize the gradient descent process.

Interpretability: Normalized features are more interpretable. When features are on a similar scale, it's easier to 
compare the relative importance of each feature in making predictions.

In the code example provided in the previous response, we used Min-Max scaling, a common form of normalization that
transforms features to a range between 0 and 1. Other normalization techniques, such as z-score normalization
(standardization), can also be used, depending on the characteristics of the data and the requirements of the 
deep learning model.

Overall, normalization is a best practice in machine learning and deep learning to ensure that your model can
learn effectively and make accurate predictions, especially when dealing with features that have varying scales.

'''



### NO NEWS DATASET YET


KeyError: 'Date'

In [16]:

#LSTM RNN GRU PROPHET ARIMA GARCH

'''
STEPS TO BE DONE BEFORE DATA WRANGLING,

This code calculates and adds Moving Averages (SMA), RSI, Stochastic Oscillator (%K and %D), and Bollinger Bands
(Upper and Lower) as additional features to your cryptocurrency price data.
 After adding these features, you can proceed with data preprocessing, splitting, and model building as demonstrated in previous responses.

'''
import pandas as pd
import numpy as np
import yfinance as yf  # For downloading cryptocurrency price data

# Download cryptocurrency price data from Yahoo Finance (e.g., Bitcoin)
crypto_data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Calculate Moving Averages (e.g., 50-day and 200-day)
crypto_data['SMA_50'] = crypto_data['Close'].rolling(window=50).mean()
crypto_data['SMA_200'] = crypto_data['Close'].rolling(window=200).mean()

# Calculate RSI (Relative Strength Index)
def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

crypto_data['RSI'] = calculate_rsi(crypto_data)

# Calculate Stochastic Oscillator
def calculate_stochastic_oscillator(data, window=14):
    low_min = data['Low'].rolling(window=window).min()
    high_max = data['High'].rolling(window=window).max()

    k = 100 * ((data['Close'] - low_min) / (high_max - low_min))
    d = k.rolling(window=3).mean()
    
    return k, d

crypto_data['%K'], crypto_data['%D'] = calculate_stochastic_oscillator(crypto_data)

# Calculate Bollinger Bands
def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()

    upper_band = rolling_mean + (rolling_std * num_std_dev)
    lower_band = rolling_mean - (rolling_std * num_std_dev)
    
    return upper_band, lower_band

crypto_data['Upper_Band'], crypto_data['Lower_Band'] = calculate_bollinger_bands(crypto_data)

# Drop rows with NaN values (resulting from rolling calculations)
crypto_data.dropna(inplace=True)

# Now, crypto_data DataFrame contains the original price data along with calculated indicators

# You can use this data for training and testing your cryptocurrency price prediction model.
# Further preprocessing, splitting, and model building steps would be similar to previous examples.




[*********************100%%**********************]  1 of 1 completed


In [18]:
crypto_data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'SMA_50',
       'SMA_200', 'RSI', '%K', '%D', 'Upper_Band', 'Lower_Band'],
      dtype='object')

In [19]:
crypto_data.value_counts()

Open          High          Low           Close         Adj Close     Volume       SMA_50        SMA_200       RSI        %K         %D         Upper_Band    Lower_Band  
9151.183594   9230.983398   9100.824219   9159.040039   9159.040039   12252601475  9394.715820   8534.130850   51.256905  38.456920  36.030095  9394.314926   9028.320035     1
34290.292969  35364.925781  33881.835938  35350.187500  35350.187500  20856685287  34332.460938  44702.747041  55.060698  99.754524  91.568809  35530.362655  30070.851798    1
41501.480469  41847.164062  41038.097656  41441.164062  41441.164062  19652846215  42986.917031  49074.233018  79.802026  95.313459  81.800377  43086.622475  33966.390025    1
41501.746094  42126.300781  40961.097656  41374.378906  41374.378906  27819532341  42130.383828  48060.225000  42.041989  51.436065  39.248710  47375.466998  37616.579486    1
41551.269531  44092.601562  41444.582031  43790.894531  43790.894531  31141681925  46570.229609  45271.428047  40.454855  44.

In [20]:

##############   LSTM


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load cryptocurrency price data
#data = pd.read_csv('crypto_price_data.csv')  # Replace with your data file
data = crypto_data  # Replace with your data file

# Preprocess data
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Feature scaling
scaler = MinMaxScaler()
data['Price'] = scaler.fit_transform(data['Price'].values.reshape(-1, 1))

# Split data into training and testing sets
train_size = int(len(data) * 0.8)
train_data, test_data = data.iloc[:train_size], data.iloc[train_size:]

# Create sequences for training
def create_sequences(data, seq_length):
    sequences = []
    target = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target_val = data.iloc[i+seq_length]
        sequences.append(seq)
        target.append(target_val)
    return np.array(sequences), np.array(target)

seq_length = 10  # You can adjust this sequence length
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Build an LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Inverse transform predictions
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)

# Calculate RMSE (Root Mean Squared Error)
train_rmse = np.sqrt(mean_squared_error(train_data.iloc[seq_length:], train_predictions))
test_rmse = np.sqrt(mean_squared_error(test_data.iloc[seq_length:], test_predictions))

print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')

# Visualize predictions
plt.figure(figsize=(12, 6))
plt.plot(data.index[seq_length:len(train_data)], train_predictions, label='Train Predictions', color='blue')
plt.plot(data.index[len(train_data) + seq_length:], test_predictions, label='Test Predictions', color='red')
plt.plot(data.index, data['Price'], label='Actual Price', color='green')
plt.legend()
plt.show()

2023-09-06 22:24:18.967671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 22:24:19.143992: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-06 22:24:19.170717: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-06 22:24:19.170728: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

KeyError: 'Date'

In [ ]:

##############   RNN


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam

# Load cryptocurrency price data
data = pd.read_csv('crypto_price_data.csv')  # Replace with your data file

# Preprocess data
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Feature scaling
scaler = MinMaxScaler()
data['Price'] = scaler.fit_transform(data['Price'].values.reshape(-1, 1))

# Split data into training and testing sets
train_size = int(len(data) * 0.8)
train_data, test_data = data.iloc[:train_size], data.iloc[train_size:]

# Create sequences for training
def create_sequences(data, seq_length):
    sequences = []
    target = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target_val = data.iloc[i+seq_length]
        sequences.append(seq)
        target.append(target_val)
    return np.array(sequences), np.array(target)

seq_length = 10  # You can adjust this sequence length
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Build an RNN model
model = Sequential()
model.add(SimpleRNN(units=50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Inverse transform predictions
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)

# Calculate RMSE (Root Mean Squared Error)
train_rmse = np.sqrt(mean_squared_error(train_data.iloc[seq_length:], train_predictions))
test_rmse = np.sqrt(mean_squared_error(test_data.iloc[seq_length:], test_predictions))

print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')

# Visualize predictions
plt.figure(figsize=(12, 6))
plt.plot(data.index[seq_length:len(train_data)], train_predictions, label='Train Predictions', color='blue')
plt.plot(data.index[len(train_data) + seq_length:], test_predictions, label='Test Predictions', color='red')
plt.plot(data.index, data['Price'], label='Actual Price', color='green')
plt.legend()
plt.show()

In [ ]:

##########   GRU MODEL

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam

# Load cryptocurrency price data
data = pd.read_csv('crypto_price_data.csv')  # Replace with your data file

# Preprocess data
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Feature scaling
scaler = MinMaxScaler()
data['Price'] = scaler.fit_transform(data['Price'].values.reshape(-1, 1))

# Split data into training and testing sets
train_size = int(len(data) * 0.8)
train_data, test_data = data.iloc[:train_size], data.iloc[train_size:]

# Create sequences for training
def create_sequences(data, seq_length):
    sequences = []
    target = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target_val = data.iloc[i+seq_length]
        sequences.append(seq)
        target.append(target_val)
    return np.array(sequences), np.array(target)

seq_length = 10  # You can adjust this sequence length
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Build a GRU model
model = Sequential()
model.add(GRU(units=50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Inverse transform predictions
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)

# Calculate RMSE (Root Mean Squared Error)
train_rmse = np.sqrt(mean_squared_error(train_data.iloc[seq_length:], train_predictions))
test_rmse = np.sqrt(mean_squared_error(test_data.iloc[seq_length:], test_predictions))

print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')

# Visualize predictions
plt.figure(figsize=(12, 6))
plt.plot(data.index[seq_length:len(train_data)], train_predictions, label='Train Predictions', color='blue')
plt.plot(data.index[len(train_data) + seq_length:], test_predictions, label='Test Predictions', color='red')
plt.plot(data.index, data['Price'], label='Actual Price', color='green')
plt.legend()
plt.show()

In [ ]:

'''
#################   FACEBOOK PROPHET

In this code:

We load the cryptocurrency price data into a Pandas DataFrame, ensuring that the date column is named 'ds' and
the price column is named 'y'.

We initialize a Prophet model and fit it to the data.

We create a dataframe with future dates for which we want to make predictions. You can adjust the periods parameter
to specify how far into the future you want to forecast.

We use the predict method to generate predictions for the future dates.

Finally, we plot the forecasted prices using Matplotlib.

Keep in mind that Prophet is a powerful tool for time series forecasting and can handle various types of time series
data.
 You can further fine-tune the model by adding holidays, adjusting seasonality settings, and working with additional
 regressors if you have relevant data.

 '''


import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt

# Load cryptocurrency price data
data = pd.read_csv('crypto_price_data.csv')  # Replace with your data file

# Preprocess data
data['Date'] = pd.to_datetime(data['Date'])
data = data.rename(columns={'Date': 'ds', 'Price': 'y'})

# Initialize and fit the Prophet model
model = Prophet()
model.fit(data)

# Create a dataframe with future dates for prediction
future = model.make_future_dataframe(periods=365)  # You can adjust the prediction horizon

# Make predictions
forecast = model.predict(future)

# Plot the forecast
fig = model.plot(forecast)
plt.title('Cryptocurrency Price Forecast')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:

############   ARIMA


from statsmodels.tsa.arima_model import ARIMA

# Fit ARIMA model
model = ARIMA(data, order=(p, d, q))
results = model.fit()

# Make predictions
forecast, stderr, conf_int = results.forecast(steps=n_steps)


'''
ARIMA (AutoRegressive Integrated Moving Average) Model:
Data Preprocessing:

Load your cryptocurrency price data and convert it into a time series format. Ensure the data is stationary 
(i.e., the mean and variance do not vary significantly over time) by differencing or 
other transformations if necessary.
Exploratory Data Analysis (EDA):

Examine your time series data using plots like time series plots,
 autocorrelation plots (ACF), and partial autocorrelation plots (PACF) to determine the order of differencing (d),
 autoregressive (p), and moving average (q) terms.
Model Fitting:

Fit an ARIMA model to your preprocessed data using the determined values of p, d, and q. You can use libraries like 
statsmodels in Python for this purpose.
Model Evaluation:
Evaluate the ARIMA model using metrics like Mean Squared Error (MSE), Root Mean Squared Error (RMSE), or others
appropriate for your use case.
Prediction:
Use the trained ARIMA model to make future price predictions.
'''


import pandas as pd
import numpy as np
import yfinance as yf
from statsmodels.tsa.arima_model import ARIMA

# Download cryptocurrency price data from Yahoo Finance (e.g., Bitcoin)
crypto_data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Preprocess the data and make it stationary (if necessary)
# For example, you can difference the data to remove trend
crypto_data['Diff'] = crypto_data['Close'].diff()

# Split the data into training and testing sets
train_data = crypto_data['Diff'][:'2022-01-01']
test_data = crypto_data['Diff']['2022-01-01':]

# Fit an ARIMA model
model = ARIMA(train_data, order=(p, d, q))  # Define appropriate values for p, d, and q
model_fit = model.fit(disp=0)

# Make predictions
n_forecast = len(test_data)
forecast, stderr, conf_int = model_fit.forecast(steps=n_forecast)

# Evaluate the model's performance
# You can use metrics like Mean Squared Error (MSE), Root Mean Squared Error (RMSE), etc.

In [ ]:

######  GARCH



#GARCH (Generalized Autoregressive Conditional Heteroskedasticity) Model:
#Data Preprocessing:

#Similarly, preprocess your cryptocurrency price data and ensure it is stationary.
#Exploratory Data Analysis (EDA):

#Examine the data for signs of volatility clustering or ARCH (Autoregressive Conditional Heteroskedasticity) effects.
#Model Fitting:
#Fit a GARCH model to your preprocessed data. You can use libraries like arch in Python.



from arch import arch_model

# Fit GARCH model
model = arch_model(data, vol='Garch', p=p, q=q)
results = model.fit()

# Forecast volatility
forecast_volatility = results.conditional_volatility[-n_steps:]


'''
Model Evaluation:

Evaluate the GARCH model using appropriate metrics. GARCH models typically focus on modeling volatility,
so you might want to assess how well it captures volatility patterns.
Volatility Forecasting:

Use the trained GARCH model to forecast future volatility, which can be useful for risk management in 
cryptocurrency trading.
'''


In [ ]:

####  EXPONENTIAL SMOOTHING
#Exponential Smoothing models, such as Holt-Winters, are suitable for time series data with trend and seasonality. To use Exponential Smoothing:


import pandas as pd
import numpy as np
import yfinance as yf
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Download cryptocurrency price data from Yahoo Finance (e.g., Bitcoin)
crypto_data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Preprocess the data if necessary
# For example, you can remove outliers or fill missing values

# Split the data into training and testing sets
train_data = crypto_data[:'2022-01-01']
test_data = crypto_data['2022-01-01':]

# Fit an Exponential Smoothing model
model = ExponentialSmoothing(train_data['Close'], seasonal='add', seasonal_periods=7)  # Adjust parameters as needed
model_fit = model.fit()

# Make predictions
n_forecast = len(test_data)
forecast = model_fit.forecast(steps=n_forecast)

# Evaluate the model's performance
# Use appropriate metrics such as MSE, RMSE, etc.

In [25]:
import yfinance as yf
msft = yf.Ticker("MSFT")
print(msft.info)

{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software—Infrastructure', 'industryDisp': 'Software—Infrastructure', 'sector': 'Technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and power automate; and on-premises ERP and CRM applications. The Intelligent Cl